In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

from trading_system_copy import TradingSystem
from trading_optimization_copy import OptimizeTradingSystem


In [49]:
hub1_name = "ttf"
hub2_name = "the"
validation_size = 250
test_size = 250
window_size = 5
model = "tvecm_t1"

In [50]:
ots = OptimizeTradingSystem(hub1_name, hub2_name, model, validation_size, test_size, window_size)

In [51]:
best_profit, best_rolling_window, best_lower_threshold, best_upper_threshold = ots.study(rolling_window_range=[5,60,5],
                                                                                         lower_threshold_range=[0.0,2,0.1], 
                                                                                         upper_threshold_range=[100,101,1], 
                                                                                         criteria="profit", 
                                                                                         min_trades=10, 
                                                                                         verbose=True, 
                                                                                         plot=True)


Best rolling window: 10.00, Best lower threshold: 0.10, Best upper threshold: 100.00
Profit: 37.48

Mean returns: 0.25%
Standard deviation of returns: 0.74%
Sharpe ratio: 0.33
Mean returns with transaction costs: 0.25%
Confidence interval (returns): 0.16% - 0.34%
Confidence interval (returns with transaction costs): 0.15% - 0.34%

Win rate for returns: 54.40%
No trade rate for returns: 12.40%
Loss rate for returns: 33.20%

Win rate for returns with transaction costs: 54.40%
No trade rate for returns with transaction costs: 12.40%
Loss rate for returns with transaction costs: 33.20%


In [52]:
ts = TradingSystem(hub1_name, hub2_name, model, test_size, window_size)
ts.run_trading_system(rolling_window=best_rolling_window, lower_threshold=best_lower_threshold, upper_threshold=best_upper_threshold, naive=False, verbose=True, plot=True)

Profit: 11.13

Mean returns: 0.18%
Standard deviation of returns: 0.60%
Sharpe ratio: 0.30
Mean returns with transaction costs: 0.18%
Confidence interval (returns): 0.10% - 0.25%
Confidence interval (returns with transaction costs): 0.10% - 0.25%

Win rate for returns: 56.40%
No trade rate for returns: 10.00%
Loss rate for returns: 33.60%

Win rate for returns with transaction costs: 56.40%
No trade rate for returns with transaction costs: 10.00%
Loss rate for returns with transaction costs: 33.60%


In [53]:
print("Window_size", window_size)
print("Rolling_window", best_rolling_window)
print("Threshold", best_lower_threshold)

mean = f"{ts.get_returns_stats()[0]:.2f}"
print("Mean: ", mean)

std = f"{ts.get_returns_stats()[1]:.2f}"
print("Standard Deviation: ", std)

CI = ts.get_returns_stats()[3]
CI_f = f"{CI[0]:.2f}% -{CI[1]:.2f}%"
print("Confidence Interval: ", CI_f)

trade_rate = f"{(1- ts.get_trade_rates()['no_trade_rate_returns'])*100:.1f}"
print("Trade rate: ", trade_rate)

data = {
    "Pair": f"{hub1_name} - {hub2_name}",
    "Window_size": window_size,
    "Rolling_window": best_rolling_window,
    "Threshold": best_lower_threshold,
    "Mean": mean,
    "Standard_Deviation": std,
    "Confidence_Interval": CI_f,
    "Trade_rate": trade_rate
}

# Convert the dictionary into a pandas dataframe
df = pd.DataFrame([data])



Window_size 5
Rolling_window 10
Threshold 0.1
Mean:  0.18
Standard Deviation:  0.60
Confidence Interval:  0.10% -0.25%
Trade rate:  90.0


In [54]:
df

,Pair,Window_size,Rolling_window,Threshold,Mean,Standard_Deviation,Confidence_Interval,Trade_rate
0,ttf - the,5,10,0.1,0.18,0.60,0.10% -0.25%,90.0
